<a href="https://colab.research.google.com/github/malijarkas/ECSE551/blob/main/Personal_Mini_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount drive and import data

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

input_data_path = '/content/drive/My Drive/ECSE 551 Data Set/Train.csv'
# test_data_path = '/content/drive/My Drive/ECSE 551 Data Set/Test.csv'
drive.mount('/content/drive')

input_data = pd.read_csv(input_data_path, header=None)
# test_data = pd.read_csv(test_data_path, header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Helper Functions

In [ ]:
# writes string array to file with delimiter '\n'
def write_to_csv(output, file_name):
  with open(file_name, 'w') as f:
    for i in range(len(output)):
      f.write(str(output[i]))
      f.write('\n')

# Converts class names to indices, makes indexing in future operations easier
# Takes in the Y label numpy array and outputs it indexed
# The index values are dependant on the orders in class_names list
def class_name_to_index(class_names, Y_label):
    if not isinstance(Y_label, np.ndarray):
        raise TypeError("Y must be a numpy array")

    Y_string = np.copy(Y_label) #to avoid modifying Y_label
    Y_int = np.zeros(Y_string.size)

    class_dict = {name: idx for idx, name in enumerate(class_names)}
    for i in range(Y_string.size):
        if Y_string[i] in class_dict:
            Y_int[i] = class_dict[Y_string[i]]
        else:
            raise ValueError(f"Unexpected class name '{Y_string[i]}' in Y")
    return np.array(Y_int)

# Converts indices back to their class names
# Takes in the same list of class names and the numpy array Y_integer
def index_to_class_name(class_names, Y_index):
  if not isinstance(Y_index, np.ndarray):
    raise TypeError("Y_index must be a numpy array")

  Y_int = np.copy(Y_index)
  Y_string = np.empty(Y_int.size, dtype=object)

  if not (0 <= Y_int).all() and (Y_int < len(class_names)).all():
      raise ValueError("Y_index parameter contains invalid class index")

  for i in range(Y_int.size):
      Y_string[i] = class_names[int(Y_int[i])]
  return Y_string

def calculate_accuracy(predict_output, given_output):
  error_count = 0
  for i in range(predict_output.size):
    if predict_output[i] != given_output[i]:
      error_count += 1

  return error_count / predict_output.shape[0]

### Preprocess data
- try bigrams, trigrams...
- try kaggle

#### my implementation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer

num_features = 3000
class_list = ['Toronto', 'Montreal', 'London', 'Brussels']

# partition data
input_data_x = np.array(input_data[0])
input_data_y = np.array(input_data[1])

# tokenize the training data, set ngram_range
vectorizer = CountVectorizer(max_features=num_features, binary=True, ngram_range=(2, 2))

# convert input_data_x to binary
input_data_x_b = vectorizer.fit_transform(input_data_x).todense()

# map input data y labels to indices
input_data_y_indexed = class_name_to_index(np.array(class_list), input_data_y)

k_fold_test = 1

if k_fold_test == True:
  k_folds = 10
  # k_fold
  kf = KFold(n_splits=k_folds, shuffle=True, random_state=10)

  X_train = np.array([None] * k_folds)
  X_test = np.array([None] * k_folds)
  Y_train = np.array([None] * k_folds)
  Y_test = np.array([None] * k_folds)

  for i, (train_index, test_index) in enumerate(kf.split(input_data_x_b)):
    X_train[i], X_test[i] = np.array(input_data_x_b[train_index]), input_data_x_b[test_index]
    Y_train[i], Y_test[i] = input_data_y_indexed[train_index], input_data_y_indexed[test_index]
    # print(test_index[i])
    # print("--------------------")

else:
# train_test_split
  X_train = np.array([None])
  Y_train = np.array([None])
  X_test = np.array([None])
  Y_test = np.array([None])

  X_train, X_test, Y_train, Y_test = train_test_split(input_data_x_b, input_data_y_indexed, test_size=0.1, random_state=10)

#### henry's code

### alpha_Naive Bayes Classifier

TODO:
- fix the fixme
- another set of eyes to make sure its correct

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


class Naive_Bayes_Classifier:

  def __init__(self, num_features, num_classes):
    self.num_features = num_features
    self.num_classes = num_classes
    self.theta_k = np.zeros(num_classes)
    self.theta_j_k = np.zeros((num_classes, num_features))

    self.accuracy = 0;

  def train(self, X, Y, laplace_smoothing = True):
    num_features = self.num_features
    num_classes = self.num_classes

    num_samples = X.shape[0]

    class_frequency = np.zeros(num_classes)

    # k denotes the class number
    # j denotes the feature number
    theta_k = np.zeros(num_classes)
    theta_j_k = np.zeros((num_classes, num_features))

    # find frequency of output class
    for i in range (num_samples):
       for j in range (num_classes):
          class_index = int(Y[i])
          class_frequency[class_index] += 1
          break

    # find P(Y = k)
    for i in range (num_classes):
      theta_k[i] = class_frequency[i] / num_samples

    # find P(X=1|Y=k) = theta_j_k
    # P(X=0|Y=k) = 1 - P(X=1|Y=k) = 1 - theta_j_k
    for i in range (num_samples):
      for j in range (num_features):
        # FIXME: used to be X[i][0, j] until it suddenly broke? change for kfold
        if (X[i][j] == 1):
          class_index = int(Y[i])
          theta_j_k[class_index, j] += 1

    if laplace_smoothing:
      for i in range (num_classes):
        theta_j_k[i] = (theta_j_k[i] + 1) / (class_frequency[i] + 2)
    else:
      for i in range (num_classes):
        theta_j_k[i] = theta_j_k[i] / class_frequency[i]

    self.theta_k = theta_k
    self.theta_j_k = theta_j_k

  def predict(self, X):
    num_features = self.num_features
    num_classes = self.num_classes
    theta_k = self.theta_k
    theta_j_k = self.theta_j_k

    num_samples = X.shape[0]

    delta_k = np.zeros(num_classes)

    output_labels_indexed = np.zeros(num_samples, dtype=int)

    for i in range (num_samples):

      # calculate the delta_k for each class, iterating over the features
      for k in range (num_classes):
        delta_k[k] = np.log(theta_k[k])
        for j in range (num_features):
          if (X[i][0,j] == 1):
            delta_k[k] += np.log(theta_j_k[k][j])
          elif (X[i][0,j] == 0):
            delta_k[k] += np.log(1 - theta_j_k[k][j])

      #find the class with the highest delta_k
      max_delta_k_index = np.argmax(delta_k)
      output_labels_indexed[i] = int(max_delta_k_index)

    return output_labels_indexed

### main

#### kfold main


In [ ]:
from sklearn.naive_bayes import MultinomialNB


scikit_classifiers = []

scikit_accuracies = np.zeros(kf.get_n_splits())


NB_classifiers = []
output_indexed = np.array([None] * kf.get_n_splits())


for i in range (kf.get_n_splits()):
  # initialize class, train, and predict
  NB = Naive_Bayes_Classifier(num_features=3000, num_classes=4)
  NB_classifiers.append(NB)
  NB_classifiers[i].train(X_train[i], Y_train[i])

  CL = MultinomialNB()
  scikit_classifiers.append(CL)
  scikit_classifiers[i].fit(X_train[i], Y_train[i])

  output_indexed[i] = (NB.predict(X_test[i]))
  y_pred = scikit_classifiers[i].predict(np.asarray(X_test[i]))

  NB_classifiers[i].accuracy = calculate_accuracy(output_indexed[i], Y_test[i])
  scikit_accuracies[i] = calculate_accuracy(y_pred, Y_test[i])






####

In [ ]:
print("Accuracies ")
print("--------------------")
highest_accuracy_index = 0
highest_scikit_accuracy_index = 0

for i in range(len(NB_classifiers)):
  print("test " + str(i) + " : " + str(NB_classifiers[i].accuracy))
  print("scikit accuracy: " + str(scikit_accuracies[i]))
  if NB_classifiers[i].accuracy > NB_classifiers[highest_accuracy_index].accuracy:
    highest_accuracy_index = i

    if scikit_accuracies[i] > scikit_accuracies[highest_scikit_accuracy_index]:
      highest_scikit_accuracy_index = i

print("--------------------")
print("highest accuracy: " + str(NB_classifiers[highest_accuracy_index].accuracy))

print("--------------------")
print("highest scikit accuracy: " + str(scikit_accuracies[highest_scikit_accuracy_index]))

### these need to be reshuffled... thats why final predictions are low
final_label = NB_classifiers[highest_accuracy_index].predict(input_data_x_b)
final_scikit_label = scikit_classifiers[highest_scikit_accuracy_index].predict(np.asarray(input_data_x_b))
print("final prediction using highest accuracy model")
print("for my model:")
print(calculate_accuracy(final_label, input_data_y_indexed))
print("for scikit model:")
print(calculate_accuracy(final_scikit_label, input_data_y_indexed))


Accuracies 
--------------------
test 0 : 0.5571428571428572
scikit accuracy: 0.4357142857142857
test 1 : 0.5071428571428571
scikit accuracy: 0.4714285714285714
test 2 : 0.5071428571428571
scikit accuracy: 0.42857142857142855
test 3 : 0.5071428571428571
scikit accuracy: 0.42857142857142855
test 4 : 0.5142857142857142
scikit accuracy: 0.42857142857142855
test 5 : 0.5214285714285715
scikit accuracy: 0.45714285714285713
test 6 : 0.44285714285714284
scikit accuracy: 0.4
test 7 : 0.5071428571428571
scikit accuracy: 0.45714285714285713
test 8 : 0.5142857142857142
scikit accuracy: 0.4
test 9 : 0.4785714285714286
scikit accuracy: 0.4785714285714286
--------------------
highest accuracy: 0.5571428571428572
--------------------
highest scikit accuracy: 0.4357142857142857
final prediction using highest accuracy model
for my model:
0.24571428571428572
for scikit model:
0.14714285714285713


#### test_train_split main

In [ ]:
  NB = Naive_Bayes_Classifier(num_features=3000, num_classes=4)
  NB.train(X_train, Y_train)




In [ ]:
  NB.latest_output_indexed = NB.predict(X_test, Y_test)


(140,)


In [ ]:
  NB.accuracy = calculate_accuracy(NB.latest_output_indexed, Y_test)
  print(NB.accuracy)

0.32857142857142857


### Outdated

In [ ]:

# # converts class names to integers to be used as indices
# # takes in a list of class names and the numpy array Y
# def class_name_to_index(class_names, Y):
#     for i in range (Y.shape[0]):
#         for j in range (class_names.shape[0]):
#           if Y[i] == class_names[j]:
#             Y[i] = j
#             break
#           # elif j == class_names.shape[0] - 1:
#           #   print(Y[i])
#           #   print(j)
#           #   raise ValueError('unexpected class name in Y')
#     return Y

# # converts class index (i.e. integers) back to their class names
# # takes in the same list of class names and the numpy array Y_integer
# def index_to_class_name(class_names, Y_integer):
#     for i in range (Y_integer.shape[0]):
#         for j in range (class_names.shape[0]):
#           if Y_integer[i] == j:
#             Y_integer[i] = class_names[j]
#             break
#           elif j == class_names.shape[0] - 1:
#             raise ValueError('unexpected class index in Y_integer')
#     return Y_integer

# for i in range (test_data_y.shape[0]):
#   print(test_data_y[i])
#   if test_data_y[i] == 'Toronto':
#     Y_test[i] = 0
#   elif test_data_y[i] == 'Montreal':
#     Y_test[i] = 1
#   elif test_data_y[i] == 'London':
#     Y_test[i] = 2
#   elif test_data_y[i] == 'Brussels':
#     Y_test[i] = 3

